<a href="https://colab.research.google.com/github/mazen-Qaissi/Knowledge-Distillation/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


In [ ]:
# load data
batch_size = 32 

train_data = dsets.MNIST(root = './data', train = True,transform = transforms.ToTensor(), download = True)
test_data = dsets.MNIST(root = './data', train = False,transform = transforms.ToTensor())

train_gen = torch.utils.data.DataLoader(dataset = train_data,batch_size = batch_size,shuffle = True)
test_gen = torch.utils.data.DataLoader(dataset = test_data,batch_size = batch_size,shuffle = False)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3)
        self.conv3 = nn.Conv2d(32,64, kernel_size=3)
        

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        # x = F.dropout(x, p=0.5, training=self.training)
        x = F.softmax(self.conv3(x))
        # x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        return x
 
cnn = CNN()
print(cnn)

it = iter(train_gen)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
)
torch.Size([32, 576])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


In [ ]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            # print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data, float(correct*100) / float(batch_size*(batch_idx+1))))


fit(cnn,train_gen)


Epoch : 0 [0/60000 (0%)]	Loss: 6.355532	 Accuracy:0.000%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Epoch : 0 [1600/60000 (3%)]	Loss: 5.445102	 Accuracy:7.966%
Epoch : 0 [3200/60000 (5%)]	Loss: 5.445102	 Accuracy:8.540%
Epoch : 0 [4800/60000 (8%)]	Loss: 5.507602	 Accuracy:9.354%
Epoch : 0 [6400/60000 (11%)]	Loss: 5.538852	 Accuracy:9.515%
Epoch : 0 [8000/60000 (13%)]	Loss: 5.476352	 Accuracy:9.649%
Epoch : 0 [9600/60000 (16%)]	Loss: 5.507602	 Accuracy:9.759%
Epoch : 0 [11200/60000 (19%)]	Loss: 5.445102	 Accuracy:9.624%
Epoch : 0 [12800/60000 (21%)]	Loss: 5.445102	 Accuracy:9.718%
Epoch : 0 [14400/60000 (24%)]	Loss: 5.507602	 Accuracy:9.562%
Epoch : 0 [16000/60000 (27%)]	Loss: 5.476352	 Accuracy:9.662%
Epoch : 0 [17600/60000 (29%)]	Loss: 5.445102	 Accuracy:9.783%
Epoch : 0 [19200/60000 (32%)]	Loss: 5.445102	 Accuracy:9.791%
Epoch : 0 [20800/60000 (35%)]	Loss: 5.507602	 Accuracy:9.855%
Epoch : 0 [22400/60000 (37%)]	Loss: 5.382602	 Accuracy:9.874%
Epoch : 0 [24000/60000 (40%)]	Loss: 5.445102	 Accuracy:9.799%
Epoch : 0 [25600/60000 (43%)]	Loss: 5.413852	 Accuracy:9.890%
Epoch : 0 [27200/

In [ ]:
def evaluate(model):
#model = mlp
    correct = 0 
    for test_imgs, test_labels in test_gen:
        #print(test_imgs.shape)
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_gen)*32)))

evaluate(cnn)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Test accuracy:0.188% 
